[Reference](https://towardsdatascience.com/simplify-your-data-preparation-with-these-4-lesser-known-scikit-learn-classes-70270c94569f)

# 1. Pipeline: Seamlessly combine preprocessing steps

In [1]:
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

# Load diabetes dataset into pandas DataFrames
X, y = load_diabetes(scaled=False, return_X_y=True, as_frame=True)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
display(X_train.head())
display(y_train.head())

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
96,64.0,2.0,27.3,109.00,186.0,107.6,38.0,5.00,5.3083,99.0
43,54.0,1.0,24.2,74.00,204.0,109.0,82.0,2.00,4.1744,109.0
123,50.0,2.0,29.6,94.33,300.0,242.4,33.0,9.09,4.8122,109.0
23,61.0,2.0,32.0,103.67,210.0,85.2,35.0,6.00,6.1070,124.0
428,62.0,2.0,34.6,120.00,215.0,129.2,43.0,5.00,5.3660,123.0


96     150.0
43      92.0
123     84.0
23     245.0
428    310.0
Name: target, dtype: float64

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn import set_config

# Return pandas DataFrames instead of numpy arrays
set_config(transform_output="pandas")

# Build pipeline
pipe = Pipeline(steps=[
    ('impute_mean', SimpleImputer(strategy='mean')),
    ('rescale', MinMaxScaler())
])

In [3]:
# Fit the pipeline to the training data
pipe.fit(X_train)

# Transform data using the fitted pipeline
X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

In [5]:
preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler()),

])

preprocessor.fit(X_train)

X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

# 2. ColumnTransformer: Apply separate transformers to different feature subsets

In [9]:
# This code will only work if you've already run the code in the previous sections

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, MinMaxScaler
from sklearn.impute import SimpleImputer

# Categorical columns transformer - (a) impute NAs with the mode, and (b) one-hot encode
categorical_features = ['sex']
categorical_transformer = Pipeline(steps=[
    ('impute_mode', SimpleImputer(strategy='mode')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first')) # handle_unknown='ignore' ensures that any values not encountered in the training dataset are ignored (i.e. all ohe columns will be set to zero)
])

# Numerical columns transformer - (a) impute NAs with the mean, and (b) rescale
numerical_features = ['bp', 'bmi', 's1', 's2', 's3', 's4', 's5', 's6'] # All except 'age' and 'sex'
numerical_transformer = Pipeline(steps=[
    ('impute_mean', SimpleImputer(strategy='mean')),
    ('rescale', MinMaxScaler())
])

# Combine the individual transformers into a single ColumnTransformer
preprocessor = ColumnTransformer(
    
    # Chain together the individual transformers
    transformers = [
        ('categorical_transformer', categorical_transformer, categorical_features),
        ('numerical_transformer', numerical_transformer, numerical_features),
    ],
    
    # By default, columns which are not transformed by the ColumnTransformer 
    # will be dropped. By setting remainder='passthrough', we ensure that
    # these columns are retained, in their original form.
    remainder='passthrough',
    
    # Prefix feature names with the name of the transformer that generated them (optional)
    verbose_feature_names_out=True
)
# Get visual representation of the preprocessing/feature engineering pipeline
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical_transformer',
                                 Pipeline(steps=[('impute_mode',
                                                  SimpleImputer(strategy='mode')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['sex']),
                                ('numerical_transformer',
                                 Pipeline(steps=[('impute_mean',
                                                  SimpleImputer()),
                                                 ('rescale', MinMaxScaler())]),
                                 ['bp', 'bmi', 's1', 's2', 's3', 's4', 's5',
                                  's6'])])

In [10]:
# Fit the preprocessor to the training data
preprocessor.fit(X_train)

# Transform data using the fitted preprocessor
X_train_transformed = preprocessor.transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# 3. FeatureUnion: Apply multiple transformers in parallel

In [8]:
# This code will only work if you've already run the code in the previous sections

from sklearn.pipeline import FeatureUnion
from sklearn.decomposition import PCA, TruncatedSVD

# Define a feature_union object which will create reduced-dimensionality features
union = FeatureUnion(transformer_list=[
    ("pca", PCA(n_components=1)),
    ("svd", TruncatedSVD(n_components=2))
])

# Adapt the numerical transformer so that it includes the FeatureUnion
numerical_features = ['bp', 'bmi', 's1', 's2', 's3', 's4', 's5', 's6'] # All except 'age' and 'sex'
numerical_transformer = Pipeline(steps=[
    ('impute_mean', SimpleImputer(strategy='mean')),
    ('rescale', MinMaxScaler()),
    ('reduce_dimensionality', union)
])

# Categorical columns transformer - same as above
categorical_features = ['sex']
categorical_transformer = Pipeline(steps=[
    ('impute_mode', SimpleImputer(strategy='mode')),
    ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False, drop='first')) # handle_unknown='ignore' ensures that any values not encountered in the training dataset are ignored (i.e. all ohe columns will be set to zero)
])

# Build the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers = [
        ('categorical_transformer', categorical_transformer, categorical_features),
        ('numerical_transformer', numerical_transformer, numerical_features),
    ],
    remainder='passthrough',
    verbose_feature_names_out=True
)
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('categorical_transformer',
                                 Pipeline(steps=[('impute_mode',
                                                  SimpleImputer(strategy='mode')),
                                                 ('ohe',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore',
                                                                sparse=False))]),
                                 ['sex']),
                                ('numerical_transformer',
                                 Pipeline(steps=[('impute_mean',
                                                  SimpleImputer()),
                                                 ('rescale', MinMaxScaler()),
                                                 ('reduce_dimensionality',
                                                  FeatureUnion(transformer_list=[('pca',
                                                                                  PCA(n_components=1)),
                                                                                 ('svd',
                                                                                  TruncatedSVD())]))]),
                                 ['bp', 'bmi', 's1', 's2', 's3', 's4', 's5',
                                  's6'])])

# 4. FunctionTransformer: Seamlessly integrate feature engineering

In [11]:
from sklearn.preprocessing import FunctionTransformer

def add_features(X):
    X['feature_1_2'] = X['feature_1'] + X['feature_2']
    return X

def subtract_features(X):
    X['feature_3_4'] = X['feature_3'] - X['feature_4']
    return X

# Put into a pipeline
feature_engineering = Pipeline(steps=[
    ('add_features', FunctionTransformer(add_features)),
    ('subtract_features', FunctionTransformer(subtract_features))
])

In [12]:
def add_subtract_features(X):
    X['feature_1_2'] = X['feature_1'] + X['feature_2'] # Add features
    X['feature_3_4'] = X['feature_3'] - X['feature_4'] # Subtract features
    return X

# Put into a pipeline
feature_engineering = Pipeline(steps=[
    ('add_subtract_features', FunctionTransformer(add_subtract_features)),
])

In [13]:
# Combine preprocessing and feature engineering in a single pipeline
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('feature_engineering', feature_engineering),
])

pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('categorical_transformer',
                                                  Pipeline(steps=[('impute_mode',
                                                                   SimpleImputer(strategy='mode')),
                                                                  ('ohe',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['sex']),
                                                 ('numerical_transformer',
                                                  Pipeline(steps=[('impute_mean',
                                                                   SimpleImputer()),
                                                                  ('rescale',
                                                                   MinMaxScaler())]),
                                                  ['bp', 'bmi', 's1', 's2',
                                                   's3', 's4', 's5', 's6'])])),
                ('feature_engineering',
                 Pipeline(steps=[('add_subtract_features',
                                  FunctionTransformer(func=<function add_subtract_features at 0x7fd0da865360>))]))])

In [15]:
# Fit the preprocessor to the training data
pipe.fit(X_train)

# Transform data using the fitted preprocessor
X_train_transformed = pipe.transform(X_train)
X_test_transformed = pipe.transform(X_test)

In [16]:
import joblib

# Save pipeline
joblib.dump(pipe, "pipe.pkl")

# Assume that the below steps are applied in another notebook/script

# Load pipeline
pretrained_pipe = joblib.load("pipe.pkl")

# Apply pipeline to a new dataset, X_test_new
X_test_new_transformed = pretrained_pipe.transform(X_test_new)